## Compare images
**Pairwise image comparison demosntrating diverged analysis.**

This analysis cuts _n_ sub-images from a a source image based on user specified Region of Interests (ROIs). Performs all pair-wise comparisons and cluster images based on color similarity. When a single ROI changes (is being dragged), quibbler knows to only make the calculation needed to cut this specific image (see a single reporting of: "Cutting image") and re-performs the pairwise comparisons of this image with all others (2 _n_ - 1 recalculations. See reporting of"Comparing images"). 

* **Features**
    * Diverged calculations of quib slices
    * Calling user function with np.vectorize
    * Graphics-driven assignments
    * Inverse assignments
    * Assignment template


* **Try me**
    * Drag the cyan diamond marker to choose a squared number.
    * Drag the cyan square corner.

In [ ]:
import numpy as np
from functools import partial
from matplotlib import pyplot as plt, widgets
from mpl_toolkits.axes_grid1 import ImageGrid

from pyquibbler import iquib, override_all, q
from scipy.sparse.csgraph import connected_components
override_all()

%matplotlib tk

In [ ]:
@partial(np.vectorize, signature='(4),()->()', pass_quibs=True, evaluate_now=True)
def create_roi(roi, axs):
    rectprops = dict(facecolor='k', edgecolor='k', alpha=0.2, fill=True)
    widgets.RectangleSelector(axs, extents=roi, rectprops=rectprops)


@partial(np.vectorize, signature='(w,h,c),(4),()->()', otypes=[object], update_type='drop')
def cut_image(image, roi, image_num):
    print("Cutting image #{}".format(image_num))
    return image[roi[2]:roi[3], roi[0]:roi[1]]


@partial(np.vectorize, otypes=[float], update_type='drop')
def image_distance(img1, img2, image_num1, image_num2):
    print("Comparing images #{}-#{}".format(image_num1, image_num2))
    averages = np.average(img1, axis=(0, 1)) - np.average(img2, axis=(0, 1))
    return np.linalg.norm(averages) / np.linalg.norm(np.full(averages.shape, 255))


@partial(np.vectorize,  signature='(),()->()', evaluate_now=True, update_type='drop', otypes=[type(None)])
def plot_image_label(ax, index):
    ax.text(0.5, 0.5, chr(index+65), ha='center', fontsize=24, transform = ax.transAxes)
    

@partial(np.vectorize, signature='(),()->()', evaluate_now=True, update_type='drop')
def show_image(ax, img):
    ax.imshow(img)


@partial(np.vectorize, evaluate_now=True, update_type='drop')
def show_adjacency(axs, x, y, adjacent):
    symbol = 'x' if adjacent else '.'
    axs.plot(x, y, symbol, color='r')

In [3]:
# Read and draw source image
file_name = iquib('../data_files/red-white-fruits.jpg')
image = plt.imread(file_name)

plt.figure(1, figsize=[8, 7])
ax1 = plt.axes([0.1, 0.2, 0.8, 0.7])
ax1.imshow(image);

In [4]:
images_count = iquib(6)
images_count.set_assignment_template(0, 10, 1)

image_nums = np.arange(images_count);

roi_default = iquib([[20, 250, 20, 250]])
roi_default._allow_overriding = False

rois = np.repeat(roi_default, images_count, axis=0)
rois.set_assignment_template(0, 3000, 1)
rois.set_allow_overriding(True)

similiarity_threshold = iquib(np.array([.1]))

In [5]:
cut_images = cut_image(image, rois, image_nums)

In [6]:
create_roi(rois, ax1);

In [7]:
widgets.Slider(
    ax=plt.axes([0.25, 0.05, 0.65, 0.03]),
    label=q("Image count ".format, images_count),
    valmin=1, valmax=9, valstep=1,
    valinit=images_count);

In [8]:
# Figure 2 - Plot images
fig = plt.figure(2)
grid_axes = iquib(ImageGrid(fig, 111, nrows_ncols=(3, 3), axes_pad=0.1))

In [9]:
show_image(grid_axes[:images_count], cut_images);

Cutting image #0
Cutting image #1
Cutting image #2
Cutting image #3
Cutting image #4
Cutting image #5


In [10]:
# Figure 3 - Compare sub images
image_distances = image_distance(np.expand_dims(cut_images, 1), cut_images, np.expand_dims(image_nums, 1), image_nums)
adjacents = image_distances < similiarity_threshold

In [11]:
# Plot distance matrix
fig = plt.figure(3)
fig.clf()
axs = fig.add_axes([0.1, 0.15, 0.7, 0.7])
axs.imshow(1 - image_distances, cmap='gray', vmin=0, vmax=1).setp(update_type='drop')
axs.axis([-0.5, images_count - 0.5, -0.5, images_count - 0.5])
axs.set_title('pairwise distance between images')
axs.set_xlabel('Image number')
axs.set_ylabel('Image number')
show_adjacency(axs, np.expand_dims(np.arange(images_count), 1), np.arange(images_count), adjacents);

# colormap
axclr = fig.add_axes([0.85, 0.15, 0.06, 0.7])
clrmap = np.linspace(1,0,10).reshape(10,1)
axclr.imshow(clrmap, cmap='gray', vmin=0, vmax=1)
axclr.plot([-0.5,0.5], similiarity_threshold*10-0.5+np.array([0,0]), '-r', linewidth=4, picker=True)
axclr.set_xticks([])
axclr.set_yticks([])
axclr.set_ylabel('Similarity Threshold');

Comparing images #0-#0
Comparing images #0-#1
Comparing images #0-#2
Comparing images #0-#3
Comparing images #0-#4
Comparing images #0-#5
Comparing images #1-#0
Comparing images #1-#1
Comparing images #1-#2
Comparing images #1-#3
Comparing images #1-#4
Comparing images #1-#5
Comparing images #2-#0
Comparing images #2-#1
Comparing images #2-#2
Comparing images #2-#3
Comparing images #2-#4
Comparing images #2-#5
Comparing images #3-#0
Comparing images #3-#1
Comparing images #3-#2
Comparing images #3-#3
Comparing images #3-#4
Comparing images #3-#5
Comparing images #4-#0
Comparing images #4-#1
Comparing images #4-#2
Comparing images #4-#3
Comparing images #4-#4
Comparing images #4-#5
Comparing images #5-#0
Comparing images #5-#1
Comparing images #5-#2
Comparing images #5-#3
Comparing images #5-#4
Comparing images #5-#5


In [12]:
# add cluster label
image_cluster = q(connected_components, adjacents).setp(update_type='drop')
image_cluster = image_cluster[1]
plot_image_label(grid_axes[:images_count], image_cluster);

Cutting image #0
Cutting image #1
Cutting image #2
Cutting image #3
Cutting image #4
Cutting image #5
Cutting image #6
Cutting image #7
Comparing images #0-#0
Comparing images #0-#1
Comparing images #0-#2
Comparing images #0-#3
Comparing images #0-#4
Comparing images #0-#5
Comparing images #0-#6
Comparing images #0-#7
Comparing images #1-#0
Comparing images #1-#1
Comparing images #1-#2
Comparing images #1-#3
Comparing images #1-#4
Comparing images #1-#5
Comparing images #1-#6
Comparing images #1-#7
Comparing images #2-#0
Comparing images #2-#1
Comparing images #2-#2
Comparing images #2-#3
Comparing images #2-#4
Comparing images #2-#5
Comparing images #2-#6
Comparing images #2-#7
Comparing images #3-#0
Comparing images #3-#1
Comparing images #3-#2
Comparing images #3-#3
Comparing images #3-#4
Comparing images #3-#5
Comparing images #3-#6
Comparing images #3-#7
Comparing images #4-#0
Comparing images #4-#1
Comparing images #4-#2
Comparing images #4-#3
Comparing images #4-#4
Comparing ima